<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E8%B6%85%E8%B6%8A%E5%AE%A1%E6%A0%B8%E8%BE%B9%E7%95%8C%E7%9A%84embedding%E6%8A%BD%E5%8F%96%E5%92%8Cjsonl%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

对几个超越审核边界的人物进行抽取

implemented by 李鲁鲁

- [x] 对接一下bge_zh_small
- [ ] 解析单个的审核文本
- [ ] 对text字段进行base编码
- [ ] 重写一个外部的package role函数
- [ ] (opt*) 抽取bge_small的同时抽取luotuo

原版程序在

https://github.com/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E5%B0%9D%E8%AF%95%E8%BD%AC%E5%8C%96RoleLLM%E4%B8%BAHaruhi%E6%A0%BC%E5%BC%8F.ipynb

和


https://github.com/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/chatbot%E7%9A%84embedding%E6%8A%BD%E5%8F%96%E5%92%8Cjsonl%E7%94%9F%E6%88%90.ipynb



先解析文本

建立txt specific file-role name的映射

In [ ]:
specific_fname2role = {"艾希2":"艾希"}

In [ ]:
input_folder = "/content/input/"

texts = []

import os

save_names = []

#read txt files from folder
for fname in os.listdir(input_folder):
    if fname.endswith(".txt"):
        with open(os.path.join(input_folder, fname), 'r', encoding='utf-8') as f:
            texts.append(f.read())
            save_names.append(fname[:-4])

In [ ]:
print(fname[:-4])

安莉娜


In [ ]:
print(texts[0][:150])

I want you to act like 女贤者 from 情色小说 Erotica.
Please try to answer with similar language style from the Erotica.

女贤者没有性相关的经验，谈论相关的事情会觉得很羞耻
女贤者是个洁白无瑕的


In [ ]:
role_datas = []

for text in texts:
    chunks = text.split("###")
    for chunk in chunks:
        if "###" in chunk:
            print(chunk)
    role_data = {
        "system_prompt":chunks[0].strip(),
        "texts":chunks[1:]
    }
    role_datas.append(role_data)

In [ ]:
print(save_names)

['贤者', '艾希2', '艾希', '安莉娜']


In [ ]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev


/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 929, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 929 (delta 159), reused 162 (delta 109), pack-reused 693
Receiving objects: 100% (929/929), 106.26 MiB | 14.89 MiB/s, done.
Resolving deltas: 100% (489/489), done.
/content/Haruhi-2-Dev


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
!pip -q install openai tiktoken langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from ChatHaruhi.utils import float_array_to_base64, base64_to_float_array

这里我们希望用两种embeddings去写

In [ ]:
from ChatHaruhi.utils import get_bge_zh_embedding, luotuo_openai_embedding
# for english character should use get_bge_en_embedding instead

In [ ]:
embed_bge = get_bge_zh_embedding("测试")
print(embed_bge[:5])
print(len(embed_bge))

In [ ]:
embed_luotuo = luotuo_openai_embedding("测试")
print(embed_luotuo[:5])
print(len(embed_luotuo))

[-0.011614263989031315, -0.007930075749754906, 0.01195011381059885, -0.011979658156633377, -0.03503050282597542]
1536


In [ ]:
embed_funcs = [luotuo_openai_embedding, get_bge_zh_embedding]
column_names = ["luotuo_openai", "bge_zh_s15"]

TODO: 把这两个移动到util里面去

In [ ]:
import base64
import struct

def string_to_base64(text):
    byte_array = b''
    for char in text:
        num_bytes = char.encode('utf-8')
        byte_array += num_bytes

    base64_data = base64.b64encode(byte_array)
    return base64_data.decode('utf-8')

def base64_to_string(base64_data):
    byte_array = base64.b64decode(base64_data)
    text = byte_array.decode('utf-8')
    return text

In [ ]:

from tqdm import tqdm
def package_role( system_prompt, texts, embed_funcs, embed_names , if_enc = False):
    datas = []
    if True:
        data = {}
        if if_enc:
            data['enc_text'] = string_to_base64(system_prompt)
        else:
            data['text'] = system_prompt

        for embed_name in embed_names:
            data[embed_name] = 'system_prompt'
        datas.append(data)

    if True:
        data = {}

        if if_enc:
            data['enc_text'] = string_to_base64("Reserve Config Setting Here")
        else:
            data['text'] = "Reserve Config Setting Here"

        for embed_name in embed_names:
            data[embed_name] = 'config'
        datas.append(data)

    for text in tqdm(texts):
        data = {}
        text = text.strip()
        if if_enc:
            data['enc_text'] = string_to_base64(text)
        else:
            data['text'] = text
        for embed_func, embed_name in zip(embed_funcs, embed_names):
            current_vec = embed_func( text )
            encode_vec = float_array_to_base64(current_vec)
            data[embed_name] = encode_vec
        datas.append(data)
    return datas

In [ ]:
print(role_datas[0].keys())

dict_keys(['system_prompt', 'texts'])


In [ ]:
packaged_datas = package_role(
    role_datas[0]['system_prompt'],
    role_datas[0]['texts'][:2],
    embed_funcs,
    column_names,
    if_enc = True
)

100%|██████████| 2/2 [00:00<00:00,  2.10it/s]


In [ ]:


import json

def write_jsonl(datas, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for data in datas:
            json_str = json.dumps(data, ensure_ascii=False)
            f.write(json_str+"\n")

for data, save_name in zip(role_datas,save_names):
    save_path = "/content/output/" + save_name + ".jsonl"
    packaged_data = package_role(
        data['system_prompt'],
        data['texts'],
        embed_funcs,
        column_names,
        if_enc=True
    )
    write_jsonl(packaged_data, save_path)

100%|██████████| 24/24 [00:10<00:00,  2.39it/s]
